In [ ]:
import requests
import os
import matplotlib.pyplot as plt
import pandas as pd
import plotly
import plotly.express as px
import io
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold,StratifiedKFold
from sklearn.linear_model import LogisticRegression

In [ ]:
# fonction de téléchargement des données sur les accidents corporels
def telecharge(url_data,filename, path):
    # Vérifie si le dossier 'data' existe, sinon le crée
    if not os.path.exists(path):
        os.makedirs(path)
    #télécharge les données avec l'url
    response = requests.get(url_data)
    if response.status_code == 200:
        file_path = os.path.join(path, filename)
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f"Fichier {filename} téléchargé avec succès:{file_path}")
    else:
        print(f"Echec de téléchargement pour {filename}. Statut: {response.status_code}")

# API pour accéder à l'url de téléchargement
url_root="https://www.data.gouv.fr/api/1/datasets/53698f4ca3a729239d2036df/resources/"
urls={
    "usagers-2023.csv":"68848e2a-28dd-4efc-9d5f-d512f7dbe66f",
    "vehicules-2023.csv":"146a42f5-19f0-4b3e-a887-5cd8fbef057b",
    "lieux-2023.csv":"8bef19bf-a5e4-46b3-b5f9-a145da4686bc",
    "caract-2023.csv":"104dbb32-704f-4e99-a71e-43563cb604f2"
}
path='data'

for filename, resource_id in urls.items():
    url=url_root+resource_id
    response1=requests.get(url)
    if response1.status_code==200:
        data=response1.json()
        url_data=data['url']
    else:
        print("downloading failed")
    telecharge(url_data,filename,path)


In [ ]:
# base des usagers 
df_usagers = pd.read_csv("data/usagers-2023.csv", sep = ';')
df_usagers.head()

In [ ]:
# Base des véhicules 
df_vehicules = pd.read_csv("data/vehicules-2023.csv", sep = ';')
df_vehicules.head()

In [ ]:
# Base des lieux 
df_lieux = pd.read_csv("data/lieux-2023.csv", sep = ';')
df_lieux.head()

In [ ]:
df_caract = pd.read_csv("data/caract-2023.csv", sep = ';')
df_caract.head()

In [ ]:
df_merge = df_usagers.merge(df_vehicules, on=["Num_Acc","id_vehicule","num_veh"], how="inner") 
df_merge = df_merge.merge(df_lieux, on="Num_Acc", how="inner")
df_merge = df_merge.merge(df_caract, on="Num_Acc", how="inner")
df_merge.shape

### Modélisation

In [ ]:
# Packages nécéssaire
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV, Lasso
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold,StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler


In [ ]:
# calcul de l'âge des piétons
df_merge["Age"] = 2023-df_merge["an_nais"]
# Distribution de l'âge
df_merge["Age"].hist()

In [ ]:
#supression des variables d'identification
var=["Num_Acc","jour","an","com","adr","lat","long","voie","v1","v2","id_vehicule","num_veh","id_usager","an_nais", "dep"]
Num_acc=df_merge["Num_Acc"]
df_merge.drop(var, axis=1,inplace=True)

# pourcentage des valeurs manquantes par variables 
df_merge.isna().sum()[df_merge.isna().sum()!=0]/len(df_merge)*100

In [ ]:
df_merge.drop(["occutc","lartpc"], axis=1, inplace=True)

In [ ]:
# imputation des valeurs manquantes de l'âge
sns.boxplot(data=df_merge, x=df_merge['Age'])

In [ ]:
df_merge["Age"]=df_merge["Age"].fillna(df_merge["Age"].median())

In [ ]:
#recodage des variables trajet et actp de la base usager
df_merge["trajet"]=df_merge["trajet"].replace(0, value=-1)
df_merge["actp"]=df_merge["actp"].replace(0, value=-1)

# variables de type object de la base
df_merge.select_dtypes("object").columns

In [ ]:
# conversion des variables de type object en variables numériques
def replace_pr(val):
    if len(val.split())==1:
        val=val
    else:
        if len(val.split())>1:
            val=val.split()[1]
        else:
            val="-1"
    return val
df_merge["pr"]=df_merge["pr"].apply(func=replace_pr)
df_merge["pr"]=df_merge["pr"].astype(float)
df_merge["pr1"]=df_merge["pr1"].apply(func=replace_pr)
df_merge["pr1"]=df_merge["pr1"].astype(float)
def rep_virgule(val):
    val=val.replace(",",".")
    return val
df_merge["larrout"]=df_merge["larrout"].apply(rep_virgule)
df_merge["larrout"]=df_merge["larrout"].astype(float)
def heure(val):
    val=val.split(":")[0]
    return val
df_merge["hrmn"]=df_merge["hrmn"].apply(func=heure)
df_merge["hrmn"]=df_merge["hrmn"].astype(float)

df_merge["nbv"]=df_merge["nbv"].str.strip()
df_merge["nbv"]=df_merge["nbv"].replace("#VALEURMULTI","-1")
df_merge["nbv"]=df_merge["nbv"].astype(float)

df_merge["actp"]=df_merge["actp"].replace(["A","B"],["10","11"])
df_merge["actp"]=df_merge["actp"].astype(float)

In [ ]:
#suppression des colonnes qui ont plus de 20% de valeurs non renseignés
diction=[]
for column in df_merge.columns:
    if ((len(df_merge[df_merge[column]==-1][column])/df_merge.shape[0]*100)>=20) == True:
        diction.append(column)
        
df_merge=df_merge.drop(diction, axis=1)

In [ ]:
df=df_merge[["secu1","secu2","catu","place","grav"]]
df=df[df["secu2"]!=-1]

In [ ]:
df_merge.isnull().sum()[df_merge.isnull().sum()!=0]/len(df_merge)*100

In [ ]:
import numpy as np
# on remplace toute les valeurs non renseignées des variables restantes par le mode
diction2=[]
for column in df_merge.columns:
    if len(df_merge[df_merge[column]==-1][column])/df_merge.shape[0]!=0:
        diction2.append(column)

for col in diction2:
    #mode=df_merge[col].value_counts().idxmax()
    df_merge[col]=df_merge[col].replace(-1,np.nan)


In [ ]:
df_merge["place"].unique()

#### Relation target_variable

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency
import numpy as np

for col in df.columns:

    # Création d'une table de contingence
    contingency_table = pd.crosstab(df[col], df["grav"])

    # Calcul du chi-deux et du V de Cramer
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    n = contingency_table.sum().sum()
    mindim = min(contingency_table.shape)-1

    # Calcul du V de Cramer
    v = np.sqrt(chi2 / (n * mindim))
    print(col)
    print('Coefficient cramer:', v)
    print('P-value:', p)

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency
import numpy as np

for col in df_merge.drop(["Age",'hrmn','vma',"nbv","grav"], axis=1).columns:

    # Création d'une table de contingence
    contingency_table = pd.crosstab(df_merge[col], df_merge["grav"])

    # Calcul du chi-deux et du V de Cramer
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    n = contingency_table.sum().sum()
    mindim = min(contingency_table.shape)-1

    # Calcul du V de Cramer
    v = np.sqrt(chi2 / (n * mindim))
    print(col)
    print('Coefficient cramer:', v)
    print('P-value:', p)

In [ ]:
from scipy.stats import pearsonr
for col in ["Age",'hrmn','vma',"nbv"]:
    #dfcol=df[[col,"grav"]].dropna()
    corr, p_value = pearsonr(df_merge[col], df_merge["grav"])
    print(col)
    print('Coefficient de corrélation de point bisériel:', corr)
    print('P-value:', p_value)

In [ ]:
#On caractérise chaque accident par le niveau de gravité le plus haut
df_merge["Num_Acc"]=Num_acc
import numpy as np
id_acc=df_merge["Num_Acc"].unique()
index_true=[]
for i in id_acc:
    df_id=df_merge[df_merge["Num_Acc"]==i]
    n_grav=np.max(df_id["grav"])
    index=df_merge[(df_merge["Num_Acc"]==i) & (df_merge["grav"]==n_grav)].index[0]
    #if len(index)!=0:
    #df_merge.drop(index=index, inplace=True)
    index_true.append(index)
        
df_merge.drop(["Num_Acc"], axis=1, inplace=True)

In [ ]:
# Niveau de gravité de l'accident présente dans la nouvelle base
df_merge=df_merge.loc[index_true]
df_merge["grav"].value_counts()

Dans les accidents enregistrés, aucun n'a laisser toutes les personnes impliquées indemnes.

On recode la variable "gravité" par: 
- 2: niveau de gravité **grave** (2)
- 3: niveau de gravité **moyen** (1)
- 4: niveau de gravité **faible** (0)

In [ ]:
#recodage de la gravité de l'accident
df_merge["grav"]=df_merge["grav"].replace([3,4], value=[1,0])
# distribution de la gravité
df_merge["grav"].value_counts()/df_merge.shape[0]*100

#### Sélection des variables

Pour la selection des variables qui seront utilisées dans le modèle de régression logistique, nous utiliserons une regression Lasso. C'est une technique de régularisation qui consiste à appliquer une pénalité pour éviter le surapprentissage et améliorer la précision des modèles statistiques.


In [ ]:
#recodage des variables 
#df_merge["trajet"]=df_merge["trajet"].replace(0, value=-1)
#df_merge["actp"]=df_merge["actp"].replace(0, value=-1)

df_merge["catv"] = df_merge["catv"].replace([4,30,32,2,5,31,33,34],value=1)
df_merge["catv"] = df_merge["catv"].replace([36,35,3],value=2)
df_merge["catv"] = df_merge["catv"].replace([16,17,20,21],value=3)
df_merge["catv"] = df_merge["catv"].replace([7,8,9],value=4)
df_merge["catv"] = df_merge["catv"].replace([10,11,12],value=5)
df_merge["catv"] = df_merge["catv"].replace([13,14,15],value=7)
df_merge["catv"] = df_merge["catv"].replace([16,17],value=8)
df_merge["catv"] = df_merge["catv"].replace([18,37,39,19,40,38],value=9)
df_merge["catv"] = df_merge["catv"].replace([41,42,43],value=10)
df_merge["catv"] = df_merge["catv"].replace([50,60],value=11)

df_merge["manv"] = df_merge["manv"].replace(2,value=1)
df_merge["manv"] = df_merge["manv"].replace(12,value=11)
df_merge["manv"] = df_merge["manv"].replace(14,value=13)
df_merge["manv"] = df_merge["manv"].replace(16,value=15)
df_merge["manv"] = df_merge["manv"].replace(18,value=17)
df_merge["manv"] = df_merge["manv"].replace([20,21,22,23,24,25,26],value=19)

df_merge["obs"] = df_merge["obs"].replace([4,5],value=3)
df_merge["obs"] = df_merge["obs"].replace(15,value=12)
df_merge["obs"] = df_merge["obs"].replace([8,11],value=7)
df_merge["obs"] = df_merge["obs"].replace(10,value=6)
df_merge["obs"] = df_merge["obs"].replace(16,value=14)





In [ ]:
#corrélation entre les variables
#plt.figure(figsize=(20,20))
#sns.heatmap(df_merge.corr(),annot=True)

In [ ]:
# on retire les variables qui ont une forte corrélation avec d'autres
#df_merge=df_merge.drop(["place","agg"], axis=1)

Par la suite, nous utilisons un encodage onehot sur les variables catégorielles (catégorielles nominales) qui va creer de nouvelles variables indicatrices et vont faciliter les interprétations des résultats du modèle. Aussi, nous standardisons les variables continues.

In [ ]:
# en utilisant une regression lasso
numeric_var=["Age",'hrmn','vma',"nbv"]
lasso_x=df_merge.drop(["grav"],axis=1)
categorical_var=lasso_x.drop(numeric_var, axis=1).columns
lasso_y=df_merge["grav"]

numeric_pipeline = Pipeline(
    steps=[("scale", StandardScaler())]
)

categorical_pipeline = Pipeline(
    steps=[
        ("one-hot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
    ]
)

Nous déterminons ensuite le meilleur hyperparamètre alpha en utilisant une validation croisée afin que les variables qui seront sélectionner par le Lasso prédisent au mieux le niveau de gravité des accidents.

In [ ]:
base=categorical_pipeline.fit_transform(lasso_x.drop(columns=numeric_var,axis=1))
base_cont=numeric_pipeline.fit_transform(lasso_x[numeric_var])
data=pd.DataFrame(data=base, columns=categorical_pipeline.get_feature_names_out())
data[numeric_var]=base_cont

my_alphas = np.array([0.001, 0.01, 0.02, 0.025])
lcv = LassoCV(alphas=my_alphas, fit_intercept=False, random_state=0, cv=3).fit(
    data, lasso_y)
print("alpha optimal :", lcv.alpha_)

Implémentons enfin le Lasso, et conservons la base de données avec les variables non pénalisées.

In [ ]:
# Application du lasso
model = Lasso(fit_intercept=False, alpha=lcv.alpha_)
lasso_optimal=model.fit(data, lasso_y)
data.columns[np.abs(lasso_optimal.coef_)>0] # variables sélectionner

In [ ]:
data_model=data[data.columns[np.abs(lasso_optimal.coef_)>0]]

#### Implémentation de la régression logistique

Division de la base de données en données d'entrainement et données de test 

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(data_model,lasso_y, test_size=0.2, random_state=42)

In [ ]:
params={'penalty':["l2",None], 'solver':["newton-cg"]}
cv=RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid = GridSearchCV(LogisticRegression(multi_class="multinomial",class_weight="balanced"), params, cv=cv)
grid.fit(x_train, y_train)
print("best params:",grid.best_params_)
print("best score:",grid.best_score_)


In [ ]:
model=grid.best_estimator_
acc_train = accuracy_score(y_train, model.predict(x_train))
acc_test = accuracy_score(y_test, model.predict(x_test))
print("Training Accuracy:", round(acc_train, 2))
print("Test Accuracy:", round(acc_test, 2))

In [ ]:
#confusion matrix
from sklearn import metrics
predictions = model.predict(x_test)
cm = metrics.confusion_matrix(y_test, predictions)
plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
all_sample_title = 'Accuracy Score: {0}'.format(acc_test)
#plt.title(all_sample_title, size = 15);
#plt.savefig('toy_Digits_ConfusionSeabornCodementor.png')

In [ ]:
importances1 = model.coef_[0]
odds_ratios1 = pd.Series(np.exp(importances1), index=data_model.columns).sort_values() #coefficient des variables dans le modèle
fig1 = px.bar(x=odds_ratios1.tail(15),y=odds_ratios1.tail(15).index, orientation="h",title="Odds Ratio-reference 1")
fig1.update_layout(xaxis_title="exp(coef)", yaxis_title="Variables")


In [ ]:
importances2 = model.coef_[1]
odds_ratios2 = pd.Series(np.exp(importances2), index=data_model.columns).sort_values() #coefficient des variables dans le modèle
fig2 = px.bar(x=odds_ratios2.tail(15),y=odds_ratios2.tail(15).index, orientation="h",title="Odds Ratio-reference 2")
fig2.update_layout(xaxis_title="exp(coef)", yaxis_title="Variables")

- L'utilisation de la ceinture de sécurité seule augmente la chance que la ravité de l'accident soit moyenne de 82%.
- Lorsque la voiture se trouve entre deux files avant l'accident, la gravité de l'accident à 54% d'être moyenne que d'être faible
- un véhicule scooter<50cm3, l'autoroute, heurté une glissière en béton augmentent la chance d'avoir un accident de gravité moyenne de 54%,51%,47% respectivement.
- une collision de trois véhicules et plus-en chaine augmentent la chance d'avoir un accident de gravité moyenne de 45%.
 - les caractéristiques suivantes: une manoeuvre d'évitement, un accident sur piste cyclable, heurté un véhicule en stationnement, avec un scooter > 50cm3 et <=125cm3, avoir un accident sur la chaussée, ou avec un véhicule léger, ou encore sous une pluie légère,  augmentent la chance d'avoir un accident de gravité moyenne entre 15% et 30%. 

 - Avoir un accident sur un cyclomoteur<50cm3 ou sur une motocyclette augmente la probabilité que l'accident soit grave de plus de 40%.

In [ ]:
fig3 = px.bar(x=odds_ratios1.head(10),y=odds_ratios1.head(10).index, orientation="h",title="Odds Ratio-reference 1")
fig3.update_layout(xaxis_title="exp(coef)", yaxis_title="Variables")


In [ ]:
fig4 = px.bar(x=odds_ratios2.head(10),y=odds_ratios2.head(10).index, orientation="h",title="Odds Ratio-reference 2")
fig4.update_layout(xaxis_title="exp(coef)", yaxis_title="Variables")

### Implémentation d'un modèle de random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
n_estimators = [int(x) for x in np.arange(100, 500, 100)]
max_features = ['auto', 'sqrt']
max_depth = [10,15]
min_samples_split = [2, 5]
min_samples_leaf = [1, 2]
criterion = ['gini', 'entropy']
params_grid = {'n_estimators': n_estimators, 
               'max_depth': max_depth,
               'min_samples_leaf': min_samples_leaf,
               'criterion': criterion
              }

cv=RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid = GridSearchCV(RandomForestClassifier(class_weight="balanced"), params_grid, cv=cv)
grid.fit(x_train, y_train)
print("best params:",grid.best_params_)
print("best score:",grid.best_score_)